In [3]:
import struct
struct.pack('2i5s',3,4,b'aacff')

b'\x03\x00\x00\x00\x04\x00\x00\x00aacff'

In [18]:
struct.unpack('2i', b'\x03\x00\x00\x00\x04\x00\x00\x00aacff'[:8])

(3, 4)

In [17]:
struct.calcsize('2i')

8

In [33]:
import struct
import random


int_length = struct.calcsize('8i')

def make_datas():
    req_or_ack = random.randint(1, 2)
    device_id = random.randint(1, 20)
    client_id = random.randint(21, 40)
    str_datas = ''.join([chr(random.randint(97,122)) for i in range(random.randint(10,20))])
    print('data', str_datas)
    return req_or_ack, device_id, client_id, str_datas

def union_datas(req_or_ack, device_id, client_id, data, message_type=1, device_type=0, message_version=0):
    # req_or_ack req:1, ack:2
    # req_len 6th int
    # ack_len 7th int
    # data string
    
    data_len = len(data)
    datas = [message_version, device_id, device_type, client_id, message_type, req_or_ack]
    if req_or_ack == 1:
        datas.extend((data_len, 0, data.encode()))
    elif req_or_ack == 2:
        datas.extend((0, data_len, data.encode()))
    else:
        raise ValueError('req_or_ack is not valid!')
    print('primary data:', datas)
    return datas, data_len

def pack_msg_between_dev(datas, data_len):
    char_len = len(datas[-1])
    packed_data = struct.pack('8i%ds' % data_len, *datas)
    print('packed_data', packed_data)
    return packed_data

datas = make_datas()
datas, data_len = union_datas(*datas)
packed_data = pack_msg_between_dev(datas, data_len)

data felddhoailurlpq
primary data: [0, 19, 0, 29, 1, 2, 0, 15, b'felddhoailurlpq']
packed_data b'\x00\x00\x00\x00\x13\x00\x00\x00\x00\x00\x00\x00\x1d\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x0f\x00\x00\x00felddhoailurlpq'


In [34]:
def unpack_msg_between_dev(datas):
    unpack_datas = struct.unpack('8i', datas[:int_length])
    print(unpack_datas)
    req_or_ack = unpack_datas[-3]
    if req_or_ack == 1:
        data_len = unpack_datas[-2]
    elif req_or_ack == 2:
        data_len = unpack_datas[-1]
    else:
        raise ValueError('req_or_ack is not valid!')
    unpack_datas = struct.unpack('8i%ds' % data_len, datas)
    print(unpack_datas, unpack_datas[-1].decode())
    
    
unpack_msg_between_dev(packed_data)
    

(0, 19, 0, 29, 1, 2, 0, 15)
(0, 19, 0, 29, 1, 2, 0, 15, b'felddhoailurlpq') felddhoailurlpq
